In [4]:
import os
import math

import torch
import numpy as np

In [5]:
data_dirpath='D:/data/mask_rcnn'

device=torch.device("cpu")
box_coder_encoder=torch.load(os.path.join(data_dirpath, "box_coder_encoder.pt"),map_location=device, weights_only=True)
matched_gt_boxes=box_coder_encoder['matched_gt_boxes']
anchors=box_coder_encoder['anchors']
print('matched_gt_boxes ', type(matched_gt_boxes), [m.shape for m in matched_gt_boxes])
print('anchors ', type(anchors), [a.shape for a in anchors])

matched_gt_boxes  <class 'list'> [torch.Size([214830, 4]), torch.Size([214830, 4])]
anchors  <class 'list'> [torch.Size([214830, 4]), torch.Size([214830, 4])]


In [9]:
# regression_targets = self.box_coder.encode(matched_gt_boxes, anchors)
reference_boxes=matched_gt_boxes
proposals=anchors

boxes_per_image=[len(b) for b in reference_boxes]
print('boxes_per_image ', boxes_per_image)
reference_boxes=torch.cat(reference_boxes, dim=0)
print('reference_boxes ', reference_boxes.shape, reference_boxes.min(0).values,
     reference_boxes.max(0).values)
proposals=torch.cat(proposals, dim=0)
print('proposals ', proposals.shape, proposals.min(0).values, proposals.max(0).values)
# targets = self.encode_single(reference_boxes, proposals)

boxes_per_image  [214830, 214830]
reference_boxes  torch.Size([429660, 4]) tensor([ 38.,  21., 114., 298.]) tensor([248., 178., 369., 435.])
proposals  torch.Size([429660, 4]) tensor([-362., -362.,   11.,   11.]) tensor([ 881.,  945., 1194., 1258.])


In [11]:
device=reference_boxes.device
dtype=reference_boxes.dtype
weights=(1.0, 1.0, 1.0, 1.0) # weights for x, y, w, h
weights=torch.as_tensor(weights, device=device, dtype=dtype)
# targets = encode_boxes(reference_boxes, proposals, weights)

# perform some unpacking to make it JIT-fusion friendly
wx=weights[0]
wy=weights[1]
ww=weights[2]
wh=weights[3]

proposals_x1=proposals[:,0].unsqueeze(1)
proposals_y1=proposals[:,1].unsqueeze(1)
proposals_x2=proposals[:,2].unsqueeze(1)
proposals_y2=proposals[:,3].unsqueeze(1)

reference_boxes_x1=reference_boxes[:,0].unsqueeze(1)
reference_boxes_y1=reference_boxes[:,1].unsqueeze(1)
reference_boxes_x2=reference_boxes[:,2].unsqueeze(1)
reference_boxes_y2=reference_boxes[:,3].unsqueeze(1)

print('proposals_y1 ', proposals_y1.shape)
print('proposals_y2 ', proposals_y2.shape)
print('reference_boxes_x1 ', reference_boxes_x1.shape)
print('reference_boxes_x2 ', reference_boxes_x2.shape)

proposals_y1  torch.Size([429660, 1])
proposals_y2  torch.Size([429660, 1])
reference_boxes_x1  torch.Size([429660, 1])
reference_boxes_x2  torch.Size([429660, 1])


In [ ]:
# implementation starts here
ex_widths=proposals_x2-proposals_x1
ex_heights=proposals_y2-proposals_y1
ex_ctr_x=proposals_x1+0.5*ex_widths # center in x
ex_ctr_y=proposals_y1+0.5*ex_heights # center in y

gt_widths=reference_boxes_x2-reference_boxes_x1
gt_heights=reference_boxes_y2-reference_boxes_y1
gt_ctr_x=reference_boxes_x1+0.5*gt_widths
gt_ctr_y=reference_boxes_y1+0.5*gt_heights

targets_dx=wx*(gt_ctr_x-ex_ctr_x)/ex_widths
targets_dy=wy*(gt_ctr_y-ex_ctr_y)/ex_heights
targets_dw=ww*torch.log(gt_widths/ex_widths)
targets_dh=wh*torch.log(gt_heights/ex_heights)

targets=torch.cat((targets_dx, targets_dy, targets_dw, targets_dh), dim=1)